In [55]:
import csv
import math
import operator
import re
from collections import Counter, defaultdict, namedtuple
from functools import lru_cache
from itertools import product
from types import SimpleNamespace

def is_integer(num):
    return int(num) == num

def sort_string(s):
    return ''.join(sorted(s))

# Day 1: Inverse captcha

In [17]:
def captcha(digits, step=1):
    size = len(digits)
    match_next = [
        int(v) if v == digits[(k+step) % size] else 0
        for k, v in enumerate(digits)
    ]
    return sum(match_next)

with open('inputs/day1', 'r') as f:
    digits = f.read().strip()
    
captcha(digits)

1119

In [18]:
captcha(digits, step=(len(digits)//2))

1420

# Day 2: Corruption Checksum

In [37]:
def min_max_diff(iterable):
    return max(iterable) - min(iterable)

def checksum(*args, row_func=min_max_diff):
    return sum([row_func(line) for line in args])

with open('inputs/day2', 'r') as f:
    reader = csv.reader(f, delimiter="\t")
    lines = [list(map(int, line)) for line in reader]

checksum(*lines)

51139

In [40]:
def evenly_divisible(iterable):
    divs = (p[0] / p[1] for p in product(iterable, repeat=2) if p[0] != p[1])
    for num in divs:
        if is_integer(num):
            return num
        
checksum(*lines, row_func=evenly_divisible)

272.0

# Day 3: Spiral Memory
First observation is that the lower right corner number of each ring in the spiral is equal to (2n+1)^2
Knowing the ring number is one part of the Manhattan Distance.  The second part of the distance is how far the we need to walk to the center of the edge. The coordinates of the center of each edge for a ring is given by: 
((2n+1)^2 - n, (2n+1)^2 - 3n, (2n+1)^2 - 5n, (2n+1)^2 - 7n)

In [3]:
def spiral_ring_radius(num):
    radius = math.ceil((math.sqrt(num) - 1)/2)
    return int(radius)

def spiral_ring_edge_centers(ring):
    corner = ((2 * ring) + 1) ** 2
    return [
        corner - 1 * ring,
        corner - 3 * ring,
        corner - 5 * ring,
        corner - 7 * ring,
    ]
    
def spiral_manhattan_distance(num):
    ring = spiral_ring_radius(num)
    centers = spiral_ring_edge_centers(ring)
    walks = [abs(num - c) for c in centers]
    return (ring, min(walks))

day3_input = 368078
spiral_manhattan_distance(day3_input)

(303, 68)

The new spiral values can be built with a recursive function.  
F(0) = 1
F(n) = F(n-1)

# Day 4: High-Entropy Passphrase

In [13]:
def is_valid_passphrase(passphrase):
    words = passphrase.strip().split(' ')
    return len(words) == len(set(words))
    
with open('inputs/day4', 'r') as f:
    lines = list(map(str.strip, f.readlines()))

sum([is_valid_passphrase(line) for line in lines])

466

In [18]:
def is_valid_passphrase_anagrams(passphrase):
    words = [sort_string(w) for w in passphrase.strip().split(' ')]
    return len(words) == len(set(words))

sum([is_valid_passphrase_anagrams(line) for line in lines])

251

# Day 5: A Maze of Twisty Trampolines, All Alike

In [36]:
def _simple_instr_change(i):
    return 1

def _complex_instr_change(i):
    return -1 if i >= 3 else 1
    
def jump_outside_stepcount(instructions, func=_simple_instr_change):
    curr = 0
    count = 0
    while True:
        try:
            moves = instructions[curr]
        except IndexError:
            break
            
        instructions[curr] += func(instructions[curr])
        curr += moves
        count += 1
        
    return count

with open('inputs/day5', 'r') as f:
    instructions = list(map(int, f.readlines()))
    
jump_outside_stepcount(instructions.copy())

387096

In [37]:
jump_outside_stepcount(instructions.copy(), func=_complex_instr_change)

28040648

# Day 6: Memory Reallocation

In [52]:
class Memory(list):
    def __str__(self):
        return ",".join(map(str, self))
        
    def reallocate(self):
        index = self.most_blocks
        size = len(self)
        blocks, self[index] = self[index], 0
        for i in range(1, blocks + 1):
            self[(index+i) % size] += 1

    @property
    def most_blocks(self):
        return self.index(max(self)) 

def memory_reallocation(memory):       
    history = [str(memory),]   
    while True:
        memory.reallocate()     
        if str(memory) in history:
            break
        else:
            history.append(str(memory))
            
    return len(history)

with open('inputs/day6', 'r') as f:
    memory_bank = Memory(map(int, f.read().split('\t')))
    
memory_reallocation(memory_bank)

11137

In [53]:
memory_reallocation(memory_bank)

1037

# Day 7: Recursive Circus
For part 2, we need to find the program node that has balanced children but is itself unbalanced

In [51]:
class ProgramTower(defaultdict):
    def get_bottom(self):
        for name, program in self.items():
            if not program.parent:
                return name

class ProgramTowerNode:
    def __init__(self, weight=None, parent=None, supports=None):
        self.weight = weight
        self.parent = parent
        self.supports = supports if supports else []
    
    def __repr__(self):
        return f'{self.name} ({self.weight})'
    
    @property
    @lru_cache()
    def total_weight(self):
        weight = self.weight if self.weight else 0 
        return weight + sum([child.total_weight for child in self.supports])
        
DAY7_RE = r'([a-z]+) \((\d+)\)(?: -\> (.*))?'

program_tower = ProgramTower(lambda: ProgramTowerNode())
with open('inputs/day7', 'r') as f:
    for line in f:
        name, weight, supports = re.match(DAY7_RE, line).groups()
        
        program_tower[name].name = name
        program_tower[name].weight = int(weight)
        if supports:
            for support in map(str.strip, supports.split(',')):
                program_tower[name].supports.append(program_tower[support])
                program_tower[support].parent = program_tower[name]
            
program_tower.get_bottom()  


'gynfwly'

In [53]:
def find_unbalanced(tower):
    child_weights = defaultdict(list)
    for child in tower.supports:
        child_weights[child.total_weight].append(child)

    unbalanced_weight, unbalanced = next(
        filter(lambda x: len(x[1]) == 1, child_weights.items()), 
        (None, None)
    )
    if not unbalanced:
        return tower
    else:    
        return find_unbalanced(unbalanced[0])

def find_unbalanced_adjustment(node):
    weights = Counter([x.total_weight for x in node.parent.supports])
    common_weight = weights.most_common(1)[0][0]
    adjustment = (node.total_weight - common_weight)
    return node.weight - adjustment
    
find_unbalanced_adjustment(find_unbalanced(program_tower['gynfwly']))

1526

# Day 8: I Heard You Like Registers
Since all the register values initialize to 0, a Counter is a good option

In [67]:
class CpuRegister(Counter):
    MATH_OPS = {
        'inc': operator.add,
        'dec': operator.sub,
    }
    
    LOGIC_OPS = {
        '>': operator.gt,
        '>=': operator.ge,
        '<': operator.lt,
        '<=': operator.le,
        '!=': operator.ne,
        '==': operator.eq
    }
    
    def __init__(self):
        self.historical_max = 0
        
    def check(self, register, op, value):
        return self.LOGIC_OPS.get(op)(self[register], value)
    
    def process(self, register, op, value):
        self[register] = self.MATH_OPS.get(op)(self[register], value)
        self.historical_max = max(self.historical_max, self.max_value[1])
        
    @property
    def max_value(self):
        return self.most_common(1)[0]
    
DAY8_RE = r'([a-z]+) (inc|dec) ([-0-9]+) if ([a-z]+) ([<>=!]+) ([-0-9]+)'

cpu_register = CpuRegister()
with open('inputs/day8', 'r') as f:
    for line in f:
        reg, math_op, math_val, test_reg, test_op, test_val = re.match(DAY8_RE, line).groups()
        if cpu_register.check(test_reg, test_op, int(test_val)):
            cpu_register.process(reg, math_op, int(math_val))
            
cpu_register.max_value

('ic', 5215)

In [68]:
cpu_register.historical_max

6419

# Day 9: Stream Processing
We use a series of regular expressions to filter out the ! and also the garbage.  Once we are left with a stream that is only {} characters, we can compute the values.  Similarly, we can use regular expressions to extract the characters that are in the <> garbage tags

In [85]:
with open('inputs/day9', 'r') as f:
    stream = f.read().strip()
        
def stream_value(stream):
    stream = re.sub(r'!.', '', stream)
    stream = re.sub(r'<.*?>', '', stream)
        
    total = 0
    curr = 0
    for c in stream:
        if c == '{':
            curr += 1
        elif c == '}':
            total += curr
            curr -= 1
    return total    

stream_value(stream)

16689

In [86]:
def garbage_count(stream):
    stream = re.sub(r'!.', '', stream)
    clusters = re.findall(r'<(.*?)>', stream)
    return sum(list(map(len, clusters)))

garbage_count(stream)

7982

# Day 10: Knot Hash

In [87]:
class CircularList(list):
    def __getitem__(self, key):
        key = key % len(self)
        return super().__getitem__(key)
    
    def __setitem__(self, key):
        key = key % len(self)
        return super().__setitem__(key)